In [1]:
from cycada_utils import *

epoch        = 0 
n_epochs     = 100
batch_size   = 1
lr           = 0.0002
b1           = 0.5 
b2           = 0.999 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'On {device}')

KeyboardInterrupt: 

In [4]:
def sample_images(dataloader, e, i):
    """show a generated sample from the test set"""
    imgs = next(iter(dataloader))
    G_AB.eval()
    G_BA.eval()
    real_A = imgs['A'].type(Tensor) # A : monet
    fake_B = G_AB(real_A).detach()
    real_B = imgs['B'].type(Tensor) # B : photo
    fake_A = G_BA(real_B).detach()
    recona = G_BA(fake_B).detach()
    reconb = G_AB(fake_A).detach() 

    # Resize images to 10 by 10
    # Arange images along x-axis
    real_A = make_grid(real_A, nrow=5, normalize=True, scale_each=True, padding=1)
    fake_B = make_grid(fake_B, nrow=5, normalize=True, scale_each=True, padding=1)
    real_B = make_grid(real_B, nrow=5, normalize=True, scale_each=True, padding=1)
    fake_A = make_grid(fake_A, nrow=5, normalize=True, scale_each=True, padding=1)
    reconA = make_grid(recona, nrow=5, normalize=True, scale_each=True, padding=1)
    reconB = make_grid(reconb, nrow=5, normalize=True, scale_each=True, padding=1)

    # Arange images along y-axis    
    image_grid = torch.cat((real_A, fake_B, real_B, fake_A, reconA, reconB), 1)
    plt.imshow(image_grid.cpu().permute(1,2,0))
    plt.title('Real A vs Fake B | Real B vs Fake A| Recon A vs Recon B')
    plt.axis('off')
    plt.gcf().set_size_inches(10, 10) # set image size to 10 by 10 inches
    plt.savefig(os.path.join('Figure_PDFs', f'epoch_{str(e)}_iter{str(i)}.png' ))
    plt.show();

In [5]:
root='/datacommons/carlsonlab/srs108/planet_imgs'
files_train = sorted(glob.glob(os.path.join(root+'/usembassy')+'/*.*')[:])
paired_files_train = [(img, 1) for img in files_train]
random.shuffle(paired_files_train)

files_test = sorted(glob.glob(os.path.join(root+'/thil')+'/*.*')[:])
paired_files_test = [(img, 0) for img in files_test]
random.shuffle(paired_files_test)


In [6]:
len(paired_f iles_train)
len(files_test)

SyntaxError: invalid syntax (<ipython-input-6-1dd1ee9cd19a>, line 1)

In [7]:
image_path = paired_files_train[index][0]
label_tr = self.paired_files_train[index][1]
image_tr = Image.open(image_path)


NameError: name 'index' is not defined

In [ ]:
class CyDataset(Dataset):
    def __init__(self, root, transforms_=None):
        self.transform = transforms.Compose(transforms_)
        self.mode = mode

        self.files_train = sorted(glob.glob(os.path.join(root+'/usembassy')+'/*.*')[:])
        self.paired_files_train = [(img, 1) for img in self.files_train]
        random.shuffle(self.paired_files_train)
        
        self.files_test = sorted(glob.glob(os.path.join(root+'/thil')+'/*.*')[:])
        self.paired_files_test = [(img, 0) for img in self.files_test]
        random.shuffle(self.paired_files_test)

    def  __getitem__(self, index):
        
        image_path = self.paired_files_train[index][0]
        label_tr = self.paired_files_train[index][1]
        image_tr = Image.open(image_path)


        item_te = self.transform(image_te)
        label_te = self.paired_files_test[index][1]
            
        return {'img_tr':item_tr, 
                'lbl_tr': label_tr,
                'img_te':item_te,
                'lbl_te':label_te}
    
    def __len__(self): return len(self.paired_files_train)


In [8]:
tr = CyDataset(root='/datacommons/carlsonlab/srs108/planet_imgs', transforms_=transforms_, unaligned=True)
dataloader = DataLoader(
    tr,
    batch_size=1, # 1
    shuffle=True,)
val_dataloader = DataLoader(
    tr,
    batch_size=5,
    shuffle=False,
)

NameError: name 'CyDataset' is not defined

In [9]:
def discriminator_acc(prediction, target_is_real=True):
    pred_labels = torch.round(torch.sigmoid(prediction)).squeeze()
    true_labels = torch.ones_like(pred_labels) if target_is_real else torch.zeros_like(pred_labels)

    correct_predictions = torch.sum(pred_labels == true_labels).item()
    total_predictions = true_labels.size(0) 

    accuracy = correct_predictions / total_predictions

    return accuracy

In [10]:
import functools
norm_layer = functools.partial(nn.BatchNorm2d, affine=True, track_running_stats=True)
G_AB = ResnetGenerator(3, 3, 64, norm_layer=norm_layer, use_dropout=False, n_blocks=9)
G_BA = ResnetGenerator(3,3,64, norm_layer=norm_layer, use_dropout=False, n_blocks=9)

D_A = NLayerDiscriminator(3)
D_B = NLayerDiscriminator(3)
D_ft = FeatureDiscriminator()

C_A = Feature_Extractor()
C_B = LeNet(3)

G_AB.to(device)
G_BA.to(device)
D_A.to(device)
D_B.to(device)
D_ft.to(device)
C_A.to(device)
C_B.to(device)

ganloss = GANLoss().to(device)                         #use to fool discriminator
cycleloss = torch.nn.L1Loss().to(device)               #difference between reconstructed img and original
identityloss = torch.nn.L1Loss().to(device)            #difference between generator output from input img and input img
clsloss = torch.nn.CrossEntropyLoss().to(device)       #difference between domain classifications between input img and generator output

optimizer_G = torch.optim.Adam(itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=lr, betas=(b1,b2))
optimizer_D = torch.optim.Adam(itertools.chain(D_A.parameters(), D_B.parameters()), lr=lr, betas=(b1, b2))
optimizer_C_A = torch.optim.Adam(C_A.parameters(), lr=lr, betas=(b1, 0.999))    
optimizer_C_B = torch.optim.Adam(C_B.parameters(), lr=lr, betas=(b1, 0.999))    #different learning rate??
optimizer_D_ft = torch.optim.Adam(D_ft.parameters(), lr=lr/10, betas=(b1, 0.999))


G_AB.apply(weights_init_normal)
G_BA.apply(weights_init_normal)
D_A.apply(weights_init_normal)
D_B.apply(weights_init_normal)
D_ft.apply(weights_init_normal)
C_A.apply(weights_init_normal)
C_B.apply(weights_init_normal)

print()

In [11]:
Tensor      = torch.cuda.FloatTensor
G_loss      = []
D_loss      = []
D_accs      = []
Dft_loss    = []
num_classes = 2

for e in range(n_epochs):
    for i, batch in enumerate(tqdm(dataloader)):
        real_A = batch['img_tr'].type(Tensor)
        real_B = batch['img_te'].type(Tensor)
        real_lbl = batch['lbl_tr'].type(Tensor).long()
        
        oneh = torch.eye(num_classes).to(device)
        
        source_label = torch.ones(real_A.size()[0]).long().to(device)
        target_label = torch.zeros(real_B.size()[0]).long().to(device)
        
        G_AB.train()
        G_BA.train()
        
        
          
        #Generator Forward Pass
        fake_b         = G_AB(real_A)             #used throughout
        fake_a         = G_BA(real_B)             #used only for cycle loss and discriminator A
        recov_a        = G_BA(fake_b)
        recov_b        = G_AB(fake_a)
# ---------------------------------------------------------------------------------
        #Feature Discriminator Forward Pass
        pred_source    = D_ft(C_B(fake_b.detach())) # D_ft(C_B(G_A(A)))
        pred_target    = D_ft(C_B(real_B))          # D_ft(C_B(B))
        
        dft_acc_source = discriminator_acc(pred_source, True)
        dft_acc_targ   = discriminator_acc(pred_target, False)
        
        acc_D_ft       = (dft_acc_source + dft_acc_targ)/2 #feature discriminator accuracy
        
# ----------------------------------------------------------------------------------

        optimizer_G.zero_grad()
        loss_GAN_AB, _ = ganloss(D_B(fake_b), True)                # tricking the 'fake-B' into 'real-B'
        loss_GAN_BA, _ = ganloss(D_A(fake_a), True)                # tricking the 'fake-A' into 'real-A'
        loss_GAN       = (loss_GAN_AB + loss_GAN_BA)/2
        
        # cycle loss                                
        loss_cycle_A   = cycleloss(recov_a, real_A)               # Reduces the difference between the restored image and the real image
        loss_cycle_B   = cycleloss(recov_b, real_B)
        loss_cycle     = (loss_cycle_A + loss_cycle_B)/2
        
        loss_G         = loss_GAN + (10.0*loss_cycle) # multiply suggested weight(default cycle loss weight : 10, default identity loss weight : 5)
        loss_G.backward()
        optimizer_G.step()
        
        G_loss.append(loss_G.item())
        
# ----------------------------
# Train Feature Discriminator
# ----------------------------
        optimizer_D_ft.zero_grad()
        D_ft.train()
        
        loss_real_dft, _ = ganloss(D_ft(C_B(real_B)), True)                 
        loss_fake_dft, _ = ganloss(D_ft(C_B(fake_b.detach())), False)
                
        dft_loss = (loss_real_dft + loss_fake_dft)*0.5
        dft_loss.backward()
        optimizer_D_ft.step()
        Dft_loss.append(dft_loss.item())
        
# -----------------------------------------
# Train Discriminator A and Discriminator B
# -----------------------------------------
        optimizer_D.zero_grad()
        
        loss_real_a, _ = ganloss(D_A(real_A), True)                # train to discriminate real images as real
        loss_fake_a, _ = ganloss(D_A(fake_a.detach()), False)      # train to discriminate fake images as fake
        
        loss_D_A       = (loss_real_a + loss_fake_a)/2
    
        loss_real_b, _ = ganloss(D_B(real_B), True)                 # train to discriminate real images as real
        loss_fake_b, _ = ganloss(D_B(fake_b.detach()), False)       # train to discriminate fake images as fake
        
        loss_D_B       = (loss_real_b + loss_fake_b)/2
        
        loss_D_A.backward()
        loss_D_B.backward()
        optimizer_D.step()
        
        loss_D         = (loss_D_A + loss_D_B)/2
    
        D_loss.append(loss_D.item())
        
# -----------------
# Train Classifier A  #remember you only call this on training domain bc we only have labels for that domain.
# -----------------
        optimizer_C_A.zero_grad()
        real_c        = C_A(real_A)
        fake_c        = C_A(fake_b.detach())
        sem_loss_a    = clsloss(real_c, oneh[real_lbl])                    #cross entropy loss
        sem_loss_aa   = clsloss(fake_c, oneh[real_lbl])
        sem_loss      = (sem_loss_a + sem_loss_aa)/2
        
        sem_loss.backward()
        optimizer_C_A.step()
        
# -------------------
# Train Classifier B
# -------------------

        optimizer_C_B.zero_grad()
        cb_fake       = C_B(fake_b.detach()) 
        cb_loss       = clsloss(cb_fake, oneh[real_lbl]) #purple part in paper?

        if acc_D_ft   > 0.6:
            cb_real   = D_ft(C_B(real_B))
            tgt_lbl   = torch.ones(real_B.size()[0]).long().to(device)
            losscbadv = clsloss(cb_real, tgt_lbl)
        else:
            losscbadv = 0
        loss_C_B      = cb_loss + losscbadv
        
        loss_C_B.backward()
        optimizer_C_B.step()


        
# ---------------------------------Visualization---------------------------------       
#         if (i+1) % 200 == 0:
#             with torch.no_grad():
#                 sample_images(val_dataloader,e,i)
#                 print('[Epoch %d/%d]\n[Batch %d/%d]\n[D loss : %f]\n[G loss : %f - (adv : %f, cycle : %f]'
#                         %(e+1,n_epochs,       
#                           i+1,len(dataloader),   
#                           loss_D.item(),       
#                           loss_G.item(),       
#                           loss_GAN.item(),    
#                           loss_cycle.item(), 
#                           ))

NameError: name 'dataloader' is not defined

In [ ]:
Tensor      = torch.cuda.FloatTensor
G_loss      = []
D_loss      = []
D_accs      = []
Dft_loss    = []
num_classes = 2

for e in range(n_epochs):
    for i, batch in enumerate(tqdm(dataloader)):
        real_A = batch['img_tr'].type(Tensor)
        real_B = batch['img_te'].type(Tensor)
        real_lbl = batch['lbl_tr'].type(Tensor).long()
        
        oneh = torch.eye(num_classes).to(device)
        
        source_label = torch.ones(real_A.size()[0]).long().to(device)
        target_label = torch.zeros(real_B.size()[0]).long().to(device)
        
        G_AB.train()
        G_BA.train()
        
        
          
        #Generator Forward Pass
        fake_b         = G_AB(real_A)             #used throughout
        fake_a         = G_BA(real_B)             #used only for cycle loss and discriminator A
        recov_a        = G_BA(fake_b)
        recov_b        = G_AB(fake_a)
# ---------------------------------------------------------------------------------
        #Feature Discriminator Forward Pass
        pred_source    = D_ft(C_B(fake_b.detach())) # D_ft(C_B(G_A(A)))
        pred_target    = D_ft(C_B(real_B))          # D_ft(C_B(B))
        
        dft_acc_source = discriminator_acc(pred_source, True)
        dft_acc_targ   = discriminator_acc(pred_target, False)
        
        acc_D_ft       = (dft_acc_source + dft_acc_targ)/2 #feature discriminator accuracy
        
# ----------------------------------------------------------------------------------

        optimizer_G.zero_grad()
        loss_GAN_AB, _ = ganloss(D_B(fake_b), True)                # tricking the 'fake-B' into 'real-B'
        loss_GAN_BA, _ = ganloss(D_A(fake_a), True)                # tricking the 'fake-A' into 'real-A'
        loss_GAN       = (loss_GAN_AB + loss_GAN_BA)/2
        
        # cycle loss                                
        loss_cycle_A   = cycleloss(recov_a, real_A)               # Reduces the difference between the restored image and the real image
        loss_cycle_B   = cycleloss(recov_b, real_B)
        loss_cycle     = (loss_cycle_A + loss_cycle_B)/2
        
        loss_G         = loss_GAN + (10.0*loss_cycle) # multiply suggested weight(default cycle loss weight : 10, default identity loss weight : 5)
        loss_G.backward()
        optimizer_G.step()
        
        G_loss.append(loss_G.item())
        
# ----------------------------
# Train Feature Discriminator
# ----------------------------
        optimizer_D_ft.zero_grad()
        D_ft.train()
        
        loss_real_dft, _ = ganloss(D_ft(C_B(real_B)), True)                 
        loss_fake_dft, _ = ganloss(D_ft(C_B(fake_b.detach())), False)
                
        dft_loss = (loss_real_dft + loss_fake_dft)*0.5
        dft_loss.backward()
        optimizer_D_ft.step()
        Dft_loss.append(dft_loss.item())
        
# -----------------------------------------
# Train Discriminator A and Discriminator B
# -----------------------------------------
        optimizer_D.zero_grad()
        
        loss_real_a, _ = ganloss(D_A(real_A), True)                # train to discriminate real images as real
        loss_fake_a, _ = ganloss(D_A(fake_a.detach()), False)      # train to discriminate fake images as fake
        
        loss_D_A       = (loss_real_a + loss_fake_a)/2
    
        loss_real_b, _ = ganloss(D_B(real_B), True)                 # train to discriminate real images as real
        loss_fake_b, _ = ganloss(D_B(fake_b.detach()), False)       # train to discriminate fake images as fake
        
        loss_D_B       = (loss_real_b + loss_fake_b)/2
        
        loss_D_A.backward()
        loss_D_B.backward()
        optimizer_D.step()
        
        loss_D         = (loss_D_A + loss_D_B)/2
    
        D_loss.append(loss_D.item())
        
# -----------------
# Train Classifier A  #remember you only call this on training domain bc we only have labels for that domain.
# -----------------
        optimizer_C_A.zero_grad()
        real_c        = C_A(real_A)
        fake_c        = C_A(fake_b.detach())
        sem_loss_a    = clsloss(real_c, oneh[real_lbl])                    #cross entropy loss
        sem_loss_aa   = clsloss(fake_c, oneh[real_lbl])
        sem_loss      = (sem_loss_a + sem_loss_aa)/2
        
        sem_loss.backward()
        optimizer_C_A.step()
        
# -------------------
# Train Classifier B
# -------------------

        optimizer_C_B.zero_grad()
        cb_fake       = C_B(fake_b.detach()) 
        cb_loss       = clsloss(cb_fake, oneh[real_lbl]) #purple part in paper?

        if acc_D_ft   > 0.6:
            cb_real   = D_ft(C_B(real_B))
            tgt_lbl   = torch.ones(real_B.size()[0]).long().to(device)
            losscbadv = clsloss(cb_real, tgt_lbl)
        else:
            losscbadv = 0
        loss_C_B      = cb_loss + losscbadv
        
        loss_C_B.backward()
        optimizer_C_B.step()


        
# ---------------------------------Visualization---------------------------------       
        if (i+1) % 200 == 0:
            with torch.no_grad():
                sample_images(val_dataloader,e,i)
                print('[Epoch %d/%d]\n[Batch %d/%d]\n[D loss : %f]\n[G loss : %f - (adv : %f, cycle : %f]'
                        %(e+1,n_epochs,       
                          i+1,len(dataloader),   
                          loss_D.item(),       
                          loss_G.item(),       
                          loss_GAN.item(),    
                          loss_cycle.item(), 
                          ))

In [ ]:
def gen_dis_loss(genloss, disloss, epochs, save = True, fig_name=''):
    epoch = range(epochs)
    fig, ax = plt.subplots(1,1, figsize = (6,6))   
    ax.plot(epoch, genloss, color='b', linewidth=0.5, label='Generator')
    ax.plot(epoch, disloss, color='r', linewidth=0.5, label='Discriminator')
    ax.set_xlabel('Iters')
    ax.set_ylabel('Loss')
    ax.set_title('Generator and Discriminator Loss')
    ax.legend()
    plt.show()
    if save==True:
        fig.savefig(PROJECT_ROOT_DIR+'/'+PROJECT_SAVE_DIR+'/'+fig_name+'.png', transparent=False, facecolor='white', bbox_inches='tight')

In [ ]:
gen_dis_loss(G_loss, D_loss, len(D_loss), save = False, fig_name='gdloss')